# `osometweet` crash course

- Author: Matthew R. DeVerna (🐥= [`@mdeverna2`](https://twitter.com/mdeverna2))
- Date: September 3rd, 2021

---

## Important Stuff

- [**GitHub source code**](https://github.com/osome-iu/osometweet)
- [**GitHub wiki**](https://github.com/osome-iu/osometweet/wiki)
- [**Example code**](https://github.com/osome-iu/osometweet/tree/master/examples)

---

## Crash Course Contents

#### General

- [Installation](#installation)
- [Quick start](#quick-start)
- [Authorization](#authorization)
  - [OAuth1a (user-context)](#oauth1a)
  - [OAuth2 (app-context aka w. bearer token)](#oauth2)

#### [Endpoints](#endpoints)

- [Tweet Lookup](#tweet-lookup)
- [User Lookup](#user-lookup)
  - [With user IDs](#with-ids)
  - [With usernames](#with-usernames)
- [Timelines](#timelines)
  - [User](#user-timeline)
  - [Mentions](#mentions-timeline)
  - [Specify the number of tweets returned](#num-tweets-returned)
  - [Pagination](#pagination)
- [Follows](#follows)
  - [Following](#following)
  - [Followers](#followers)
  - [Specify the number of accounts returned](#num-results-returned)
  - [Pagination](#pagination2)
- [Streaming](#streaming)
  - [Filtered streaming](#filtered-streaming)
      - [Adding filter rules](#adding-filter-rules)
      - [Retrieving filter rules](#retrieving-filter-rules)
      - [Connecting to the filtered stream endpoint](#connecting-to-the-filtered-stream-endpoint)
      - [Deleting filter rules](#deleting-filter-rules)
  - [Sampled streaming](#sampled-streaming)
- [Search]()
  - [Recent search](#recent-search)
  - [Full archive search](#full-archive-search)

#### [Fields and Expansions](#fields-and-expansions)
- [Give me everything](#give-me-everything)
- [All from one field](#all-one-field)
- [Specific fields](#specific-fields)
- [More on fields](#more-on-fields)

#### [Utility Functions](#utility-functions)
- [`pause_until`](#pause-until)
- [`chunker`](#chunker)
- [`convert_date_to_iso`](#convert-date)

#### [Wrangle Functions](#wrangle-functions)
- [`flatten_dict`](#flatten-dict)
- [`get_dict_val`](#get-dict-val)
- [`get_dict_paths`](#get-dict-paths)

---


<a id=installation></a>

## Installation

In [ ]:
# From your Jupyter notebook
!pip install osometweet

# From the command line
# pip install osometweet

<a id=quick-start></a>

## Quick start

In [ ]:
import osometweet
import os

# Initialize the OSoMeTweet object
# bearer_token = "YOUR_TWITTER_BEARER_TOKEN"
bearer_token = os.environ.get("TWITTER_BEARER_TOKEN")

oauth2 = osometweet.OAuth2(bearer_token=bearer_token)
ot = osometweet.OsomeTweet(oauth2)

# Set some test IDs (these are Twitter's own accounts)
ids2find = ["2244994945", "6253282"]

# Call the function with these ids as input
response = ot.user_lookup_ids(user_ids=ids2find)
print(response["data"])


<a id=authorization></a>

## Authorization

<a id=oauth1a></a>

### OAuth1a (user-context)

In [ ]:
import osometweet

api_key = os.environ.get("TWITTER_API_KEY")
api_key_secret = os.environ.get("TWITTER_API_KEY_SECRET")
access_token = os.environ.get("TWITTER_ACCESS_TOKEN")
access_token_secret = os.environ.get("TWITTER_ACCESS_TOKEN_SECRET")

oauth1a = osometweet.OAuth1a(
    api_key=api_key,
    api_key_secret=api_key_secret,
    access_token=access_token,
    access_token_secret=access_token_secret
)
oauth1a

<a id=oauth2></a>

### OAuth2 (app-context aka w. bearer token)

In [ ]:
bearer_token = os.environ.get("TWITTER_BEARER_TOKEN")
oauth2 = osometweet.OAuth2(
    bearer_token=bearer_token,
    manage_rate_limits=True
)
oauth2

In [ ]:
oauth2._manage_rate_limits

---
<a id=endpoints></a>
# Endpoints

<a id=tweet-lookup></a>

## Tweet Lookup

In [ ]:
tweet_ids = ['1323314485705297926', '1328838299419627525']

# Fetch the tweets information
response = ot.tweet_lookup(tweet_ids)
print(response["data"])


<a id=user-lookup></a>

## User Lookup

<a id=with-ids></a>

## with ids

In [ ]:
# Set some test IDs (these are Twitter's own accounts)
ids2find = ["2244994945", "6253282"]

# Call the function with these ids
response = ot.user_lookup_ids(ids2find)
print(response["data"])


<a id=with-usernames></a>

## with usernames

In [ ]:
# Set some test IDs (these are Twitter's own accounts)
usernames2find = ["TwitterDev", "TwitterAPI"]

# Call the function with these ids
response = ot.user_lookup_usernames(usernames2find)
print(response["data"])


<a id=timelines></a>

## Timelines

<a id=user-timeline></a>

### User

We call the function to get `@jack`'s (jack dorsey's) 10 most recent tweets

The endpoint only supports user id, so we pass the id of @jack to the method

In [ ]:
response = ot.get_tweet_timeline('12')
response["data"]

<a id=mentions-timeline></a>

### Mentions

In [ ]:
response = ot.get_mentions_timeline('12')
response["data"]

<a id=num-tweets-returned></a>

### Specifying the number of tweets returned

Often we need much more data than Twitter returns in one request. We can request up to 100 tweets at a time using the `max_results` parameter. This has large implications with respect to query limits (i.e. how many tweets you can get with the same number of requests). Here is an example:

Call the function to get `jack`'s 100 most recent followers

In [ ]:
response = ot.get_tweet_timeline('12', max_results=100)

In [ ]:
print(f"Now we have {len(response['data'])} tweets.")
print("~~~~~~~~~~~~~~~~~~~~~~")
response["data"]

<a id=pagination></a>

### Pagination

For each user ID, Twitter allows you to request up to 3,200 of the most recent tweets, and up to 800 of the most recent tweets mentioning a user. Since you can only request (at most) 100 tweets at a time, you will need to utilize the pagination_token returned in the meta-data of the response. For example, to get the 200 most recent tweets you can do the following...

In [ ]:
response = ot.get_tweet_timeline('12', max_results=100)
response.keys()

In [ ]:
response["meta"]

In [ ]:
response_2 = ot.get_tweet_timeline(
    '12',
    pagination_token=response['meta']['next_token'],
    max_results = 100
)
response_2["meta"]

In [ ]:
response_2["data"]

<a id=follows></a>
## Follows

<a id=followers></a>
### Followers

In [ ]:
# Call the function to get "jack"'s 100 most recent followers
# The endpoint only supports user id, we pass the id of @jack to the method
response = ot.get_followers('12')
response["data"]

<a id=following></a>
### Following

In [ ]:
# Call the function to get the 100 most recent accounts that jack followed
response = ot.get_following('12')
response["data"]

<a id=num-results-returned></a>
### Specifiy the number of results

Up to 1000!

In [ ]:
# Call the function to get "jack"'s 1000 most recent followers
response = ot.get_followers('12', max_results=1000)

In [ ]:
response["meta"]

In [ ]:
print(f"Now we have {len(response['data'])} accounts.")
print("~~~~~~~~~~~~~~~~~~~~~~")
response["data"]

<a id=pagination2></a>
### Pagination

In [ ]:
# Call the function to get "jack"'s 1,000 most recent followers
response = ot.get_followers('12', max_results = 1000)
response.keys()

In [ ]:
response["meta"]

In [ ]:
# Call the function again to get another 1,000 followers:
response_2 = ot.get_followers(
    '12',
    pagination_token=response['meta']['next_token'], 
    max_results = 1000
)
response_2["meta"]

---
<a id=streaming></a>
## Streaming

Twitter offers two different **streaming endpoints** to gather tweets in real-time:
1. [Filtered stream](https://developer.twitter.com/en/docs/twitter-api/tweets/filtered-stream/introduction) : The filtered stream endpoint enables developers to filter the real-time stream of public Tweets. 
    * There are also filtered stream endpoints that enable you to create and manage matching rules, and apply those rules to filter a stream of real-time Tweets that will return matching public Tweets. For example, you can request all tweets which include the word "politics" or some other string.
2. [Sampled stream](https://developer.twitter.com/en/docs/twitter-api/tweets/sampled-stream/introduction) : The sampled stream endpoint delivers a roughly 1% random sample of publicly available Tweets in real-time.

> Note: The streaming endpoints cannot be used with the Rate Limit Manager tool. Thus, during authorization the `manage_rate_limits` parameter must be set to `False`. See [Adding filter rules](#adding-filter-rules) for an example.

### Contents
- [Filtered streaming](#filtered-streaming)
  - [Adding filter rules](#adding-filter-rules)
  - [Retrieving filter rules](#retrieving-filter-rules)
  - [Connecting to the filtered stream endpoint](#connecting-to-the-filtered-stream-endpoint)
  - [Deleting filter rules](#deleting-filter-rules)
- [Sampled streaming](#sampled-streaming)
    - [Connecting to the filtered stream endpoint](#connecting-to-the-filtered-stream-endpoint)

<a id=filtered-streaming></a>
## Filtered streaming

There are three different `osometweet` methods that will help you stream real-time filtered public tweets.

|Type|`osometweet` method| Purpose | Twitter endpoint|
|----|-------------------|---------|-----------------|
|Streaming|`filtered_stream`|Connect to the stream| `GET /2/tweets/search/stream` |
|Management|`set_filtered_stream_rule`|Add or delete rules from your stream| `POST /2/tweets/search/stream/rules` |
|Management|`get_filtered_stream_rule`|Retrieve your stream's rules| `GET /2/tweets/search/stream/rules` |

To utilize the `filtered_stream` endpoint, we must first understand how to manage the _matching rules_. Matching rules are the criteria we provide to Twitter to tell them what we want them to give us.

For example, if we wanted only tweets that contain specific keywords - for example, "coronavirus" or "indiana" - we would need to create matching rules that tells Twitter to do exactly that. Here is what that looks like.

<a id=adding-filter-rules></a>
### Adding filter rules

To add filter rules, we use the `set_filtered_stream_rule` method.

In [ ]:
oauth2 = osometweet.OAuth2(
    bearer_token=bearer_token,
    manage_rate_limits=False    # <~~~ Must be set to False!!
)
ot = osometweet.OsomeTweet(oauth2)

# Add streaming rules
rules = [{"value": "coronavirus", "tag": "all coronavirus tweets"},
         {"value": "indiana", "tag": "all indiana tweets"}]
add_rules = {"add": rules}

response = ot.set_filtered_stream_rule(rules=add_rules) #<~~~ Where the magic happens!

print("API response from adding two rules:\n")
response

#### Understand adding filter rules
We highly recommend you check out Twitter's [own documentation](https://developer.twitter.com/en/docs/twitter-api/tweets/filtered-stream/integrate/build-a-rule) on how to build a rule. Also, see [Building High Quality Filters](https://developer.twitter.com/en/docs/tutorials/building-high-quality-filters) for a more in depth review.

Nonetheless, we provide a basic explanation of how adding rules works and their structure to get you up and running.

Rules are added based on a list of dictionaries with the keys: `value` and `tag`. Each dictionary in that list makes up one rule where the keys represent the below...

- `value` : The matching criteria
    - Twitter returns tweets that match this value's input. See the links above to learn about the different ways to match tweets.
- `tag` : A label for the matching rule in that dictionary
    - This doesn't affect the actual tweets that are returned, however, if you have many rules, creating simple tags can be helpful should you want to find and delete specific rules (see [Deleting filter rules](#deleting-filter-rules) for more information on this).

So the endpoint takes in something like the below (which you can see we created above in the [Adding filter rules](#adding-filter-rules) section).

```python
{'add': [
    {'value': 'coronavirus', 'tag': 'all coronavirus tweets'},
    {'value': 'indiana', 'tag': 'all indiana tweets'}
]}
```

The top-level key `add` tells Twitter that we are adding rules and feeds the list as input of what to add.

<a id=retrieving-filter-rules></a>
### Retrieving filter rules

Now, if we wanted to check that the rules added during the [Adding filter rules](#adding-filter-rules) section are actually there, we can use the `get_filtered_stream_rule` method.

We can do this like so...

In [ ]:
current_rules = ot.get_filtered_stream_rule()
print("API response when retrieving current rules:\n")
current_rules


We can see here, that our rules are included under the `data` key. The `value` and `tag` keys are included exactly as we passed them and each rule also includes a unique identifier key `id`.

> Note, these ids will be unique each time you create these rules - i.e., that is if you add the rule which matches "coronavirus", it will create a unique value for `id`. If you then delete all of your rules and recreate that exact same rule, the value for `id` will not be the same.

<a id=connecting-to-the-filtered-stream-endpoint></a>
### Connecting to the filtered stream endpoint

Now that we have successfully added some matching rules, and we are confident they are there, we can connect to the streaming endpoint and begin gathering tweets. Here is how we do that...

In [ ]:
import json

# Returns a generator
stream = ot.filtered_stream()

# Because we have a generator, we iterate over each tweet
num_tweets = 0
for tweet in stream.iter_lines():

    # Then, we read the json `tweet` object as a dictionary and select the `data`
    # Note: if it's not there for some reason, this line returns `None`...
    data = json.loads(tweet).get("data")

    # ... but if we do find data, we can then print each tweet
    if data:
        print(data)
        num_tweets +=1
    
    # Break after receiving 10 tweets
    if num_tweets > 10:
        break

<a id=deleting-filter-rules></a>
### Deleting filter rules

To delete filter rules, we use the `set_filtered_stream_rule` method again.

To delete rules, we need to provide a list of the `id`'s for each rule that we'd like to delete. So if we have a `current_rules` object that represents the above dictionary, we can collect all of the tweet ids into a list with the below line.

In [ ]:
current_rules = ot.get_filtered_stream_rule()
current_rules

In [ ]:
all_rule_ids = [rule["id"] for rule in current_rules["data"]]
all_rule_ids

In [ ]:
delete_rule = {'delete': {'ids':all_rule_ids}}
ot.set_filtered_stream_rule(rules=delete_rule)

Notice that we needed to embed the list of ids inside of a dictionary prior to passing it to the method. Just like adding filter rules, the first key of this dictionary tells Twitter what action it should be doing - i.e., `delete` tells Twitter to remove rules, based on the list of `ids` provided.

<a id=sampled-streaming></a>
## Sampled streaming

We can access the sampled streaming endpoint with the `sampled_stream` method.

<a id=connecting-to-the-sampled-stream-endpoint></a>
### Connecting to the sampled stream endpoint

As this endpoint doesn't take any matching criteria and simply returns a general 1% sample, there is much less to think about and we can begin collecting tweets from the sampled stream in the following way...

In [ ]:
# Returns a generator
stream = ot.sampled_stream()

# Because we have a generator, we iterate over each tweet
num_tweets = 0
for tweet in stream.iter_lines():
    
    # Then, we read the json `tweet` object as a dictionary and select the `data`
    # Note: if it's not there for some reason, this line returns `None`...
    data = json.loads(tweet).get("data")

    # ... but if we do find data, we can then print each tweet
    if data:
        print(data)
        num_tweets += 1
        
    # Break after receiving 10 tweets
    if num_tweets > 10:
        break

<a id=search></a>
## Search

<a id=recent-search></a>
### Recent search

In [ ]:
ot.search(query="grumpy cat")

In [ ]:
help(ot.search)

<a id=full-archive-search></a>
### Full archive search

In [ ]:
from osometweet.utils import convert_date_to_iso

start = convert_date_to_iso("2020-01-01")
end = convert_date_to_iso("2020-02-01")

print(start)

In [ ]:
response = ot.search(
    query="grumpy cat",
    start_time=start,
    end_time=end,
    full_archive_search=True,
    max_results=10,
    everything=True
)
response

In [ ]:
response.keys()

In [ ]:
response["includes"]

In [ ]:
response["errors"]

In [ ]:
response["meta"]

<a id=fields-and-expansions></a>
## Fields and expansions
<a id=give-me-everything></a>
### Give me everything

If you want all the data fields that Twitter has to offer, follow the example below.

In [ ]:
# make request
ot.tweet_lookup('1348419350370398209', everything=True)

`everything=True` works for all `osometweet` endpoints

In [ ]:
ot.user_lookup_usernames(["mdeverna2"], everything=True)

<a id=all-one-field></a>
### Get all from a specific field
You can also retrieve all elements from specific object fields. The available object fields are:
- `UserFields`
- `TweetFields`
- `MediaFields`
- `PlaceFields`
- `PollFields`

In [ ]:
import osometweet.fields as o_fields

all_user_fields = o_fields.UserFields(everything=True)
print("User fields:", all_user_fields)

ot.user_lookup_usernames(['mdeverna2'], fields=all_user_fields)

In [ ]:
all_tweet_fields = o_fields.TweetFields(everything=True)
print("Tweet fields:",all_tweet_fields)

ot.tweet_lookup('1348419350370398209', fields=all_tweet_fields)

<a id=specific-fields></a>
### Include specific fields and expansions

`OSoMeTweet` provides the flexibility to specify exactly what the API should return.

Let us use the `tweet_lookup` endpoint as an example.

Suppose we are interested in a tweet with the unique tweet ID number, `1212092628029698048`. 

In addition to the default tweet data, we also want to know:
1. When it was created (captured in the `created_at` field)
2. How popular it was (captured in the `public_metrics` field with information like retweet counts, etc.)
3. The author of the tweet (so we will need to _expand_ the `author_id` field)
4. When the author created their account (so we also need to request the `created_at` field as a user field)

To retrieve all of this information, we simply specify these specific tweet and user fields in our query.

In [ ]:
import osometweet.fields as o_fields
import osometweet.expansions as o_expansions

# Initialize the fields object
tweet_fields = o_fields.TweetFields()

# Specify the tweet fields you need
tweet_fields.fields = ['public_metrics', 'created_at']

# Initialize the expansion object
expansions = o_expansions.TweetExpansions()

# Specify the expansions you need
expansions.expansions = ["author_id"]

# Initialize the user fields object
user_fields = o_fields.UserFields()

# Specify the fields you need
user_fields.fields = ['created_at']

# make request
ot.tweet_lookup(
    tids = ['1212092628029698048'],
    fields = tweet_fields+user_fields,
    expansions=expansions
)

<a id=more-on-fields></a>
### More on fields

Twitter [supports](https://developer.twitter.com/en/docs/twitter-api/fields) fields for `user`, `tweet`, `media`, `poll`, and `place`.
You can use `UserFields`, `TweetFields`, `MediaFields`, `PollFields`, and `PlaceFields` classes to handle them, respectively.
They only contain the default fields if not specified otherwise.

You can see what optional fields are available by

In [ ]:
import osometweet.fields as o_fields

tweet_fields = o_fields.TweetFields()
tweet_fields.optional_fields

In [ ]:
tweet_fields.default_fields

You can specify the fields by

In [ ]:
tweet_fields.fields = ['public_metrics', 'created_at']
tweet_fields.fields

You can add different fields objects up to get an object that contains all the information, and pass it to the API endpoints

In [ ]:
import osometweet.fields as o_fields

tweet_fields = o_fields.TweetFields()
tweet_fields.fields = ['public_metrics', 'created_at']

user_fields = o_fields.UserFields()
user_fields.fields = ['created_at']

sum_of_fields = tweet_fields + user_fields
# OR
# sum_of_fields = sum([tweet_fields, user_fields])

print(type(sum_of_fields))
sum_of_fields

Note: We include the `user.fields` object here but it is not returned by Twitter because we do not include the `author_id` expansion. Always make sure to double-check your asking for the right information from Twitter!!

In [ ]:
ot.tweet_lookup(
    tids = ['1212092628029698048'],
    fields = sum_of_fields
)

<a id=utility-functions></a>
## Utility Functions

We also include a few utility methods which will (hopefully) make working with the new Twitter API structure a bit easier.

First, you can import the utility methods into your environment with the following code...

<a id=pause-until></a>
#### `o_utils.pause_until`
Managing time is an important aspect of gathering data from Twitter and often you'd just like to wait some specified time. This is relatively easy with the `time` module, however, it is even easier with the `pause_until()` method. Simply input the time that you would like to pause your code until, and the method handles the rest. This method can take in a `datetime` object or a Unix epoch time-stamp. For example, if you'd like to wait ten seconds, you can simple do...

In [ ]:
import osometweet.utils as o_utils
import datetime as datetime

# The below line of code takes the time at the current moment, converts it to an epoch time-stamp
# and then adds five seconds to it.
now_plus_5_with_epoch_timestamp = datetime.datetime.now().timestamp() + 5

print("timestamp:", now_plus_5_with_epoch_timestamp)

# Then we input that into the pause_until() method and your machine will
# sleep until that specific time, five seconds later
print("Time before call:",datetime.datetime.now())

o_utils.pause_until(now_plus_5_with_epoch_timestamp)

print("Time after call:",datetime.datetime.now())

If you'd like to do this with a `datetime`object, it looks like this...

In [ ]:
import osometweet.utils as o_utils
import datetime as datetime

# The timedelta method takes input in the following way...
# timedelta(days=0, seconds=0, microseconds=0, milliseconds=0, minutes=0, hours=0, weeks=0)
now_plus_5_with_datetime_object = datetime.datetime.now() + datetime.timedelta(seconds=5)

print("datetime object:", now_plus_5_with_datetime_object)

print("Time before call:",datetime.datetime.now())

o_utils.pause_until(now_plus_5_with_datetime_object)

print("Time before call:",datetime.datetime.now())

<a id=chunker></a>
#### `o_utils.chunker`
Another reality of working with Twitter data is that you are only allowed to query Twitter with a maximum number of users/tweets/whatever per endpoint. To deal with this, we created the `o_utils.chunker` method which turns a list into a list of smaller lists where the length of those smaller lists are no longer than the user indicated size. For example...

In [ ]:
from osometweet import utils as o_util
my_list = ["user1", "user2", "user3", "user4", "user5", "user6", "user7", "user8", "user9"]
chunked_list = o_util.chunker(seq=my_list, size=2)
print(chunked_list)

<a id=convert-date></a>
#### `o_utils.convert_date_to_iso`

Some of the endpoints require specific time strings to specify where to (for example) search for different tweets.

We provide the `o_utils.convert_date_to_iso` method to make this easier...

In [ ]:
from osometweet import utils as o_util

o_util.convert_date_to_iso("2020-1-1")

Can also specify the `time_format` object if we want using any of the standard `datetime` formats.

In [ ]:
o_util.convert_date_to_iso("2020", time_format="%Y")

In [ ]:
o_util.convert_date_to_iso("2020_1_1", time_format="%Y_%m_%d")

<a id=wrangle-functions></a>
## Wrangle functions

**`osometweet.wrangle`** includes a handful of low-level data processing functions that we think could be useful when wrangling your Twitter data into something easier to analyze. The idea behind these functions was to create methods that you can easily adapt to your data processing pipeline, as opposed to creating our own that you must adopt.

Below we provide simple examples of how each function works.

### Contents
- `flatten_dict`
- `flatten_dict` and Twitter data
- `get_dict_paths`
- `get_dict_val`

### Import
We can import these functions via...

In [ ]:
from osometweet.wrangle import get_dict_paths, get_dict_val, flatten_dict

<a id=flatten-dict></a>
### `flatten_dict`

This function takes a nested dictionary and "flattens" it so that the keys of each nested dictionary are concatenated into a single string, and the value is the value at the end of that key path. This function can help you simplify the complexity of a nested dictionary (like Twitter's data objects) so it is easier to manage.

Let's see what this means.

In [ ]:
# Create dictionary
dictionary = {
    "a" : 1,
    "b" : {
        "c" : 2,
        "d" : 5
    },
    "e" : {
        "f" : 4,
        "g" : 3
    },
    "h" : 3
}

In [ ]:
dictionary

#### 1. Using function as is

In [ ]:
flat_dict = flatten_dict(dictionary)
flat_dict

In [ ]:
print(dictionary.keys())
print(flat_dict.keys())

#### 2. Changing `parent_key`
This function has an available parameter called `parent_key` which helps it work. Typically, we would recommend that you do not touch this, however, here is what tinkering with this will do - should you find some use for it. 😄 

In [ ]:
# Parent key will add `parent_key` as a prefix to all keys
flatten_dict(dictionary, parent_key = "NEW")

#### 3. Changing `sep`
Another parameter, `sep`, allows you to control the string that will separate each level of the concatenated key path. As you saw above, the default is a period (i.e., '.'), however, it can be whatever you prefer.

In [ ]:
# This string is what will separate key path strings
flatten_dict(dictionary, sep = "")


🚨🚨🚨🚨

### `flatten_dict` and Twitter data

It is important to note that the `flatten_dict` function handles all nested _dictionaires_ but will stop when it reaches something other than a dictionary. What this means is for certain data objects which contain a _list_ as the value (e.g. urls and context_annotations), further processing will be needed.

To understand what this means in more detail, I've created a [walk-through](https://github.com/osome-iu/osometweet/wiki/Method:-Wrangle-Practical-Walk-through-(flatten_dict)) of one way you might process a couple of tweets using this function while keeping the above in mind.

🚨🚨🚨🚨

In [ ]:
response = ot.user_lookup_usernames(["mdeverna2"], everything=True)

In [ ]:
response["data"][0]

In [ ]:
flatten_dict(response["data"][0])

<a id=get-dict-val></a>
### `get_dict_val`

This function returns a dictionary value at the end of a key path - provided as a `list`, like those returned by `get_dict_paths`. 

Here is what this function looks like in practice.

In [ ]:
# Create dictionary
dictionary = {
    "a" : 1,
    "b" : {
        "c" : 2,
        "d" : 5
    },
    "e" : {
        "f" : 4,
        "g" : 3
    },
    "h" : 3
}

# Create key_list
key_list = ['e', 'f']

# Execute function
get_dict_val(dictionary, key_list)

#### 2. When the input `key_path` doesn't exist
It is important to know that this function does not break should you be asking it to return a value at the end of a key path that doesn't exist. Instead, it will return `None`.

In [ ]:
# Create key_list
key_list = ['b', 'k']

# Execute function
value = get_dict_val(dictionary, key_list)

# Returns NoneType because the provided path doesn't exist
type(value)


<a id=get-dict-paths></a>
### `get_dict_paths`
This function returns a **generator** that iterates over all full key paths within `dictionary`. Because Twitter often returns only the data that is present for a specific data object (for example, certain fields/expansions (see [info](https://github.com/osome-iu/osometweet/wiki/Info:-Available-Fields-and-Expansions), [our methods](https://github.com/osome-iu/osometweet/wiki/Method:-Specifying-fields-and-expansions) for more details) will only be present within a data object if there is something to return for that field/expansion), this function can help you understand what your data object actually contains.

Here is a simple example...

In [ ]:
# Create dictionary
dictionary = {
    "a" : 1,
    "b" : {
        "c" : 2,
        "d" : 5
    },
    "e" : {
        "f" : 4,
        "g" : 3
    },
    "h" : 3
}

# Call get_dict_paths
print(list(get_dict_paths(dictionary)))

### How to use `get_dict_paths` and `get_dict_val` together with Twitter data...

In [ ]:
# Here is the user object from earlier...
user_object = response["data"][0]

In [ ]:
# Get all of the paths in that dictionary 
tweet_dict_paths = list(get_dict_paths(user_object))
tweet_dict_paths

In [ ]:
# Extract only key paths that include public metrics data points
pub_metric_paths = [path for path in tweet_dict_paths if "public_metrics" in path]
pub_metric_paths

In [ ]:
for path in pub_metric_paths:
    print(
        path[1],  # print the public metric name
        get_dict_val(response["data"][0], path)    # print the value at that path
    )

### Extracting error details

It can be very useful to take advantage of the information returned in the `errors` object by Twitter. The below `user_lookup_usernames` method call tells us that the account `@realDonaldTrump` has been suspended.

In [ ]:
ot.user_lookup_usernames(usernames=["realDonaldTrump","mdeverna2"], everything=True)